In [1]:
from logging import getLogger
from logging.config import dictConfig

from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAUROC
from yaml import safe_load

from streamvigil import ADBenchDataset
from streamvigil.detectors import RAPP

In [2]:
# Logger
with open("./logging.yml", encoding="utf-8") as file:
        config = safe_load(file)
dictConfig(config)
logger = getLogger(__name__)

In [3]:
# Anomaly Detector
detector = RAPP(
    encoder_dims=[500, 450, 400, 350, 300],
    decoder_dims=[300, 350, 400, 450, 500],
    batch_norm=True,
)

In [4]:
random_state = 42

# Load dataset
train_data = ADBenchDataset(
    "../data/9_census.npz",
    train=True,
    random_state=random_state,
)
test_data = ADBenchDataset(
    "../data/9_census.npz",
    train=False,
    random_state=random_state,
)

# DataLoader
train_loader = DataLoader(train_data, batch_size=512)
test_loader = DataLoader(test_data, batch_size=256)

In [5]:
# Training
epochs = 1
logger.info("Start training the model...")
for epoch in range(epochs):
    logger.info(f"Epoch: {epoch+1}")
    for batch, (X, _) in enumerate(train_loader):
        loss = detector.train(X)

        if batch % 100 == 0:
            logger.info(f"Loss: {loss.item():>7f}")
logger.info("Completed training the model!")

2024-06-03 13:40:43,817 [INFO] __main__: Start training the model...
2024-06-03 13:40:43,818 [INFO] __main__: Epoch: 1
2024-06-03 13:40:44,197 [INFO] __main__: Loss: 0.237438
2024-06-03 13:40:45,447 [INFO] __main__: Loss: 0.013860
2024-06-03 13:40:46,694 [INFO] __main__: Loss: 0.010878
2024-06-03 13:40:47,947 [INFO] __main__: Loss: 0.009216
2024-06-03 13:40:49,186 [INFO] __main__: Loss: 0.008655
2024-06-03 13:40:49,342 [INFO] __main__: Completed training the model!


In [6]:
# Evaluation
metrics = BinaryAUROC()
logger.info("Start evaluating the model...")

for X, y in test_loader:
    scores = detector.predict(X)
    metrics.update(scores, y)

# Compute AUROC score
logger.info(f"AUROC Score: {metrics.compute()}")

logger.info("Completed the evaluation of the model!")

2024-06-03 13:40:49,346 [INFO] __main__: Start evaluating the model...
2024-06-03 13:40:53,368 [INFO] __main__: AUROC Score: 0.5409944986406074
2024-06-03 13:40:53,369 [INFO] __main__: Completed the evaluation of the model!
